# Code Python - Electre Tri 

## Introduction to the project

The company at the origin of the request is the social landlord 3F, part of the national group Action Logement. Its request concerns the renovation of three of its housing buildings located in the Lyon region. These buildings having been built in the years 2014, the company thus considered necessary to carry out renovation works for the whole of these 3 buildings. 

The company has found it difficult in the past to find out how to renovate a building in view regarding the different aspects that come into play. For example, when looking at the minimum energy loss between primary and final energy, gas appears to be the most interesting form of energy. However, when looking at the environmental impact of this form of energy, gas is badly ranked. Thus, they wish to take into account several aspects of energy renovation in this project.  

Since several options of renovation are possible and the decision is based on multiple criteria, it has been chosen that a multi-criteria analysis should be carried out.

### Electre Tri as a multi-criteria analysis 

The Electre Tri method is the multi-criteria analysis that is selected for the project. In its process, the input data for each item is normalised using thresholds and compared to difference profiles that separate categories. This method results in an optimistic and pessimistic ranking of the elements in which every actions are ranked in categories.

In our project, 28 scenarios will be compared thanks to 16 criteria.

The following code allows to execute step by step the calculations of the Electre Tri method :

In [222]:
import csv
import pandas as pd
import numpy as np
from numpy import random, vstack, empty
import math


### Import of data from csv file as a Pandas Dataframe

The input of the whole analysis is a `csv.file` containing the following informations : 
- The **mean value of the performance** of each scenario regarding each criteria (columns 0 to 27)
- The **weight** of each criteria (column 28)
- The **variance** of each criteria (column 29)
- The **6 reference profiles** : $b0, b1, b2, b3, b4$ and $b5$ (columns 30 to 35)
- The **3 thresholds** : $q$ (the indifference threshold), $p$ (the preference threshold), $v$ (the veto threshold) (columns 36 to 38)

It is imported as a dataframe `d`.<br>
Two others parameters are also defined : 
- `λ` : the **cutting threshold**
- `repet` : the **number of repetition** of the Electre Tri method desired


In [223]:
d = pd.read_csv('Input_data.csv')
λ = 0.5
repet = 1

### Monte Carlo Function

Since we study 28 scenarios according to 16 criteria, compute all the possible combinations of performance values would not be feasible. To have more robust results, our hypothesis is to use the **Monte Carlo method** to obtain data sets from distributions and use those data sets in the Electre Tri procedure. 

Monte-Carlo simulation is used in complex systems in order to estimate some operations by using random sample and statistical modeling. 
1. Pick a value from Probability Distribution Functions
2. Run the calculation multiple times: Electre Tri in our case
3. Obtain a set of results to be analyzed 

The first step involve to be given Probability Distribution Functions as inputs. For our study, all the values will be represented as normal distributions. To descrive these distributions 2 parameters are needed : 
- the mean value : `m`
- the variance : `variance`

These values are given in the `d` DataFrame given as input of the code. 

The following function allows to :
1. Thanks to the variance and mean value for each performance : create the Normal Distribution
2. Pick a random value in each of it
3. Return a DataFrame also called `d` with the random values picked 

*The DataFrame returned will also contain all the parameters initially present in the `d` DataFrame.*

In [224]:
def MCarlo(d):
    variance = d['VAR'].values
    m = d.iloc[:, 0:28].values # for each scenario : columns 0 to 27 
    v = np.abs(m * variance[:, np.newaxis]) # calculation of the variance v 
    perf = np.random.normal(m, v) # pick a random value in th normal distribution with variance : v, mean : m 
    d.iloc[:, 0:28] = perf 
    return d


### Concordance

The concordance matrix is a table that compare each pair of alternatives being considered, in our case, the sceanarios. In other words, it evaluates how well each option performs relative to the others with respect to the set of criteria. 

This function take as input the `d` DataFrame containig all the performances as well as all the others parameters and input of the method, but only the performances, the reference profiles, and the thresholds will be used.

The objective is to calculate the concordance between each pair of alternative and reference profiles and in both ways: 
- The concordance $C_j(a_i,b_k)$
- The concordance $C_j(b_k,a_i)$ <br>

*for $i$ the scenarios, $k$ the reference profiles and $j$ the criteria*

The following schema shows how the value of the corcordance is determined : 

<center>
<figure>
  <img src="concordance.jpg" width="50%" height="50%">
</figure>
</center>

*with : <br>*
- *$u_j(a_i)$ : value of the performance of the scenario $i$ in the criterion $j$*
- *$u_j(b_k)$ : value of the reference profile $k$ in the criterion $j$*

Thus, here is how the two types of concordance can be calculated in the function: <br>
<center>

$C_j(a_i,b_k) = u_j(a_i)-u_j(b_j)+p_j/p_j-q_j$<br>
$C_j(b_k,a_i) = u_j(b_j)-u_j(a_i)+p_j/p_j-q_j$<br>

</center>


If the value is higher than one it is replaced by `1`, and if it is smaller dans zero it is replaced by `0`. 

Finally, the function returns two DataFrames : 
- `dconca` : The concordance between the performances and the reference profiles $C_k(a_i,b_j)$
- `dconcb` : The concordance between the reference profiles and the performances $C_k(b_j,a_i)$


In [225]:
def conce(d):
    new_df = pd.DataFrame() #DataFrame that will contain Cj(ai,bk)
    new_df2 = pd.DataFrame() #DataFrame that will contain Cj(bk,ai)
    for sc in d.iloc[:, 0:28]: # for each scenario : columns 0 to 27
        for pr in d.iloc[:, 30:36]: # for each reference profile : columns 30 to 35
            alpha = (d[sc]-d[pr]+d[d.columns[37]])/(d[d.columns[37]]-d[d.columns[36]]) # Cj(ai,bk) calculation
            beta = (d[pr]-d[sc]+d[d.columns[37]])/(d[d.columns[37]]-d[d.columns[36]]) # Cj(bk,ai) calculation
            new_df = pd.concat([new_df, alpha], axis=1, ignore_index=True)
            new_df2 = pd.concat([new_df2, beta], axis=1, ignore_index=True)
    # replace the negative values by zero and the one higher than one by one in both DataFrames
    new_df[new_df<0]=0
    new_df[new_df>1]=1
    new_df2[new_df2<0]=0
    new_df2[new_df2>1]=1
    return new_df, new_df2

### Discordance

The discordance matrix is a matrix that is used to represent the degree of discordance between pairs of alternatives. It is typically constructed by comparing the values of each alternative on each criterion, and determining whether the difference between the values is significant enough to cause discordance. 

This function take as input the `d` DataFrame containig all the performances as well as all the others parameters and input of the method, but only the performances, the reference profiles, and the thresholds will be used.

The objective is to calculate the discordance between each pair of alternative and reference profiles and in both ways: 
- The discordance $D_j(a_i,b_k)$
- The discordance $D_j(b_k,a_i)$ <br>
*for $i$ the scenarios, $k$ the reference profiles and $j$ the criteria*

The following schema shows how the value of the discordance is determined : 

<center>
<figure>
  <img src="discordance.jpg" width="50%" height="50%">
</figure>
</center>

Thus, here is how the two types of discordance can be calculated in the function: <br>
<center>

$D_j(a_i,b_k) = u_j(b_k)-u_j(a_i)-p_j/v_j-p_j$<br>
$D_j(b_k,a_i) = u_j(a_i)-u_j(b_k)-p_j/v_j-p_j$<br>

</center>

*with : <br>*
- *$u_j(a_i)$ : value of the performance of the scenario $i$ in the criterion $j$*
- *$u_j(b_k)$ : value of the reference profile $k$ in the criterion $j$*

If the value is higher than one it is replaced by `1`, and if it is smaller dans zero it is replaced by `0`. 

The function takes as input the `d` Dataframe.
Finally, the function returns two DataFrames : 
- `ddiscoa` : The discordance between the performances and the reference profiles $D_j(a_i,b_k)$
- `ddiscob` : The discordance between the reference profiles and the performances $D_j(b_k,a_i)$

In [226]:
def disco(d):
    new_df = pd.DataFrame() # DataFrame that will contain Dj(ai,bk)
    new_df2 = pd.DataFrame() # DataFrame that will contain Dj(bk,ai)
    for sc in d.iloc[:, 0:28]:  # for each scenario : columns 0 to 27
        for pr in d.iloc[:, 30:36]: # for each reference profile : columns 30 to 35
            alpha = (d[sc]-d[pr]+d[d.columns[37]])/(d[d.columns[38]]-d[d.columns[37]]) # Dj(ai,bk) calculation
            beta = (d[pr]-d[sc]+d[d.columns[37]])/(d[d.columns[38]]-d[d.columns[37]]) # Dj(bk,ai) calculation
            new_df = pd.concat([new_df, alpha], axis=1, ignore_index=True)
            new_df2 = pd.concat([new_df2, beta], axis=1, ignore_index=True)
     # replace the negative values by zero and the one higher than one by one in both DataFrames
    new_df[new_df<0]=0
    new_df[new_df>1]=1
    new_df2[new_df2<0]=0
    new_df2[new_df2>1]=1
    return new_df, new_df2

### Global concordance

The function allows to calculate the global concordance of each scenario regarding each threshold. It takes as input the concordance matrix and the weights for each criteria. <br>

The function takes as input the weights of each criterion, located in the `d` DataFrame as well as the concordance matrix, separated into 2 DataFrames previously : `dconca` and `dconcb`. 

The objective is, for each scenario calculate the following global concordance : 

<center>

$C(a_i,b_k) = \frac {\sum_{j} C_j(a_i,b_k) * w_j}{\sum_{j} w_j}$

</center>

*with i the scenarios, j the criteria and k the reference profiles*

This function has to be used twice :
- Once taking as input `dconca` and returning as output `dgconca` : $C(a_i,b_k)$
- Once taking as input `dconcb` and returning as output `dgconcb` : $C(b_k,a_i)$








In [227]:
def global_conc(d,dconc1):
    new_df = pd.DataFrame(index=['b0', 'b1', 'b2', 'b3', 'b4', 'b5'], columns=['S1.1','S1.2','S1.3','S1.4','S2.1','S2.2','S2.3','S2.4','S3.1','S3.2','S3.3','S3.4','S4.1','S4.2','S4.3','S4.4','S5.1','S5.2','S5.3','S5.4','S6.1','S6.2','S6.3','S6.4','S7.1','S7.2','S7.3','S7.4']) 
    i = 0
    for j in range(0, len(dconc1.columns),6): # for each scenario : one line out of 6 
        # C(ai,bk) for the scenario for each reference profile
        a = sum(dconc1[j]*d[d.columns[28]])/sum(d[d.columns[28]]) 
        b = sum(dconc1[j+1]*d[d.columns[28]])/sum(d[d.columns[28]])  
        c = sum(dconc1[j+2]*d[d.columns[28]])/sum(d[d.columns[28]]) 
        dr = sum(dconc1[j+3]*d[d.columns[28]])/sum(d[d.columns[28]])  
        e = sum(dconc1[j+4]*d[d.columns[28]])/sum(d[d.columns[28]]) 
        f = sum(dconc1[j+5]*d[d.columns[28]])/sum(d[d.columns[28]]) 
        th = [a,b,c,dr,e,f]
        new_df[new_df.columns[i]]= th # add the global concordance as a new column
        i = i+1
    return new_df

### Degree of credibility

The degree of credibility evaluates if the assumption that a scenario outperforms a profile is plausible and to what extent.

The degree of credibility is calculated thanks to :
- the global concordance of each scenario with each reference profile $C(a_i,b_k)$ named `dgconca`
- the discordance matrix, separated in two DataFrames `ddiscoa` and `ddiscob`

The objective is, for each scenario, follow these steps : 

If for all the criteria $j$,   $D_j(a_i,b_k) \le C(a_i,b_k)$:
<center>

$ \delta(a_i,b_k) = C(a_i,b_k) $

</center>

Else : 

<center>

$ \delta(a_i,b_k) = C(a_i,b_k) * \prod_{j \in J } \frac{(1-D_j(a_i,b_k))}{(1-C(a_i,b_k))} $

</center>

*With J : all the criteria for whom  $D_j(a_i,b_k) \ge C(a_i,b_k)$*

The following function should be run 2 times : 
- Once considering the comparaison of performance with reference profiles $(a_i,b_k)$
    - input : `dgconca` : the global performance  $C(a_i,b_k) $ and `ddiscoa` : the discordance :  $D_j(a_i,b_k) $
    - output : `dcreda`: the credibility $ \delta(a_i,b_k)$ <br>
    

- Once considering the comparaison of reference profiles with performances $(b_k,a_i)$
    - input : `dgconcb` : the global performance  $C(b_k,a_i) $ and `ddiscoa` : the discordance :  $D_j(b_k,a_i) $
    - output : `dcredb` the credibility $ \delta(b_k,a_i)$

In [228]:
def credibility(dgconc, ddisc):
    dcred = pd.DataFrame(index=['b0', 'b1', 'b2', 'b3', 'b4', 'b5'], columns=['S1.1','S1.2','S1.3','S1.4','S2.1','S2.2','S2.3','S2.4','S3.1','S3.2','S3.3','S3.4','S4.1','S4.2','S4.3','S4.4','S5.1','S5.2','S5.3','S5.4','S6.1','S6.2','S6.3','S6.4','S7.1','S7.2','S7.3','S7.4']) 
    for j in range(0, len(ddisc.columns), 6):
        sc = int(j/6)
        degree = [0, 0, 0, 0, 0, 0]
        for pr in range(len(dcred.index)):
            verif = sum(ddisc[j+pr][c] > dgconc[dgconc.columns[sc]][pr] for c in ddisc.index)
            if verif == 0:
                degree[pr] = dgconc[dgconc.columns[sc]][pr]
            else:
                degree[pr] = (((1-ddisc[j+pr][ddisc[j+pr] > dgconc[dgconc.columns[sc]][pr]])/(1-dgconc[dgconc.columns[sc]][pr])).prod())*dgconc[dgconc.columns[sc]][pr]
        dcred[dcred.columns[sc]] = degree
    return dcred


### Over Ranking

The objective of this step is to establish preference relationships between performance and reference profiles. 
These relationships are established than the degree  of credibility determined just before and thanks to the cutting threshold $\lambda$. 
The value of this cutting threshold can vary, and it value will be discussed later. 

There are 4 types of relationships that can be established between each $a_i$ and each $b_k$
- $a_i$  `I`  $b_k$ : $a_i$  is Indifferent to  $b_k$ 
- $a_i$  `>`  $b_k$ : $a_i$  is preferred to  $b_k$ 
- $a_i$  `<`  $b_k$ : $a_i$  is not preferred to  $b_k$ 
- $a_i$  `R`  $b_k$ : $a_i$  incomparable to $b_k$ 

This is how this these relationship are determined : 


<center>
<figure>
  <img src="over_ranking_relations.jpg" width="50%" height="50%">
  <figcaption>Preference relationships</figcaption>
</figure>
</center>

The function will return a Dataframe `dranking` containing all these relations between performance and reference profiles.



In [229]:
def over_ranking_relations(creda, credb, λ):
    #initialization
    new_df = pd.DataFrame(index=['b0', 'b1', 'b2', 'b3', 'b4', 'b5'], columns=['S1.1','S1.2','S1.3','S1.4','S2.1','S2.2','S2.3','S2.4','S3.1','S3.2','S3.3','S3.4','S4.1','S4.2','S4.3','S4.4','S5.1','S5.2','S5.3','S5.4','S6.1','S6.2','S6.3','S6.4','S7.1','S7.2','S7.3','S7.4']) 
    classementa = creda.apply(lambda x: x-λ)
    classementb = credb.apply(lambda x: x-λ)
    #1 if outperformance (S), 0 if not 
    classementa[classementa > 0] = 1  
    classementa[classementa < 0] = 0  
    classementb[classementb > 0] = 1
    classementb[classementb < 0] = 0
    mask = (classementa == classementb) & (classementa == 1)
    new_df = new_df.mask(mask, "I")
    mask = (classementa == classementb) & (classementa == 0)
    new_df = new_df.mask(mask, "R")
    mask = (classementb != 0) & (classementa == 0)
    new_df = new_df.mask(mask, "<")
    mask = (classementa != 0) & (classementb == 0)
    new_df = new_df.mask(mask, ">")
    return new_df

## Sorting

The objective of the whole method is to obtain a ranking of the multiple alternatives we have for our problem. 
This method gives two types of rankings : *the pessimistic ranking and the optimistic ranking.*

A median ranking can be obtained as an average of these two rankings.

### Pessimistic sorting

The following function permits to obtain the pessimistic ranking thanks to the over ranking relationships we just established.

This is how the ranking works : <br>

The 6 reference profiles $b0, b1, b2, b3, b4$ and $b5$ delineate 5 categories : <br>
$C1, C2, C3, C4$ and $C5$, C5 being the best one and C1 the worse : 

<center>
<figure>
  <img src="pessi_sort.jpg" width="10%" height="10%">
  <figcaption><i>Pessimistic sorting<i></figcaption>
</figure>
</center>

For each scenario, these categories will be browsed from the best to the worst ( from C5 to C1 ). 
For each reference profiles encountered the credibility $ \delta(a_i,b_k)$ will be compared to the cutting threshold $\lambda$ : 
- if $ \delta(a_i,b_k) > \lambda $ : the scenario is ranked in the category with the same number as $b_k$
- if $ \delta(a_i,b_k) < \lambda $ : we continue to the next reference profile 



In [230]:
def pessimistic_sort(dranking,pessi):
    for sc in dranking: 
        step = pessi[sc] 
        for pr in reversed(range(len(dranking.index))): 
            if dranking[sc][pr] == '>' or dranking[sc][pr] == 'I':
                step[step.index[pr]] = step[step.index[pr]] +1 #classified
                break
        pessi[sc] = step 
    return pessi 

### Optimistic sorting

The following function permits to obtain the optimistic ranking thanks to the over ranking relationships we just established.

This is how the ranking works : <br>

As previously 6 reference profiles delineate 5 categories, C5 being the best one and C1 the worse : 

<center>
<figure>
  <img src="opti_sort.jpg" width="10%" height="10%">
  <figcaption><i>Optimistic sorting<i></figcaption>
</figure>
</center>

The difference is that for this ranking, for each scenario, these categories will be browsed from the worst to the best ( from C1 to C5 ). 
For each reference profiles encountered the over ranking relation will be analyzed : 
- if $a_i$ `<` $b_k$ : the scenario is ranked in the category with the same number as $b_k$
- if $a_i$ `>` $b_k$, $a_i$ `R` $b_k$ or $a_i$ `I` $b_k$ : we continue to the next reference profile 



In [231]:
def optimistic_sort(dranking,opti):
    for sc in dranking: 
        step = opti[sc] 
        for pr in (range(len(dranking.index))): 
            if dranking[sc][pr] == '<' or dranking[sc][pr] == 'R':
                step[step.index[pr]] = step[step.index[pr]] +1 #classified
                break
        opti[sc] = step
    return opti

### Electre Tri method

This final method permits to run all the previous methods in order to compute all the steps of the Electre Tri method. 
It takes as input : 
- `d` : the input Dataframe containing the performances, the weights, the variance, the reference profiles and the thresholds
- `rep` : the number of times the Electre Tri method will be run, defined at the beginning of the code



In [232]:
def electre_tri (d,rep):
    #initialization
    temp = np.zeros((5,28))
    pessi_sort = pd.DataFrame(temp, index=['C1', 'C2', 'C3', 'C4', 'C5'], columns=['S1.1','S1.2','S1.3','S1.4','S2.1','S2.2','S2.3','S2.4','S3.1','S3.2','S3.3','S3.4','S4.1','S4.2','S4.3','S4.4','S5.1','S5.2','S5.3','S5.4','S6.1','S6.2','S6.3','S6.4','S7.1','S7.2','S7.3','S7.4'])
    opti_sort = pd.DataFrame(temp, index=['C1', 'C2', 'C3', 'C4', 'C5'], columns=['S1.1','S1.2','S1.3','S1.4','S2.1','S2.2','S2.3','S2.4','S3.1','S3.2','S3.3','S3.4','S4.1','S4.2','S4.3','S4.4','S5.1','S5.2','S5.3','S5.4','S6.1','S6.2','S6.3','S6.4','S7.1','S7.2','S7.3','S7.4'])
    #repetitions
    for i in range(rep) :
        d = MCarlo(d) 
        dconca, dconcb = conce(d)
        ddisca, ddiscb = disco(d)
        dgconca = global_conc(d,dconca)
        dgconcb = global_conc(d,dconcb)
        dcreda = credibility(dgconca, ddisca)
        dcredb = credibility(dgconcb, ddiscb)
        dranking = over_ranking_relations(dcreda, dcredb, λ)
        pessi_sort = pessimistic_sort(dranking,pessi_sort)
        opti_sort = optimistic_sort(dranking,opti_sort)
    pessi_sort = pessi_sort.apply(lambda x: (x/rep)*100) #%
    opti_sort = opti_sort.apply(lambda x: x/rep*100) #%
    return opti_sort, pessi_sort, dranking, dconca, ddisca, dgconca, dcreda

The `electre_tri` function is run returning two DataFrames : `opti_sort` and `pessi_sort`

Then two csv files are created containing the repartition of the scenarios in the categories as percentages : 
- `pessimistic_sorting.csv` for the pessimistic sorting 
- `optimistic_sorting.csv` for the optimistic sorting

In [233]:
opti_sort, pessi_sort, dranking, dconca, ddisca, dgconca, dcreda = electre_tri (d, repet)
#pessi_sort.to_csv('pessimistic_sorting.csv')
#opti_sort.to_csv('optimistic_sorting.csv')

              S1.1          S1.2          S1.3          S1.4          S2.1  \
g1.1  0.000000e+00 -9.626458e+05 -1.313849e+06 -1.098185e+06 -1.141705e+06   
g1.2 -1.877270e+06 -5.782066e+05 -6.640549e+05 -5.224578e+05 -5.944837e+05   
g1.3  0.000000e+00  1.003583e+06  9.263877e+05  9.158590e+05  8.016681e+05   
g1.4 -7.388440e+04 -4.009312e+04 -3.708540e+04 -4.581734e+04 -3.649709e+04   
g1.5  0.000000e+00 -5.914287e+03 -6.435907e+03 -6.081998e+03 -5.914162e+03   
g2.1  4.333395e+00  2.612985e+00  3.215760e+00  3.064914e+00  2.763048e+00   
g2.2  9.997465e-01  1.086018e+00  9.525844e-01  1.123010e+00  1.048149e+00   
g2.3  4.381436e+00  0.000000e+00  5.901390e+00  4.882926e+00  3.625858e+00   
g2.4  3.474620e+00  3.435486e+00  3.322878e+00  3.597786e+00  5.382162e+00   
g3.1 -3.764442e+02 -2.035120e+02 -1.321285e+02 -1.791962e+02 -1.847720e+02   
g3.2  0.000000e+00  2.518736e+00  3.535851e+00  2.154765e+00  4.426585e+00   
g3.3  4.912369e+00  5.591066e+00  5.261013e+00  5.413262e+00  4.